# Affiliation Parsing

This notebook's goal is to parse the affiliation field into country, institute, address

## parse country

- split the affiliation value by comma, resulting in 20+ columns - we have no idea what each column may contain
- parse the country: 1. get a list of countries

In [1]:
import sqlite3
import pandas as pd
import timeit

In [2]:
# connect and return pandas dataframe
conn = sqlite3.connect("../data/pub_raw.db")
#authors = pd.read_sql_query("select * from author order by random() limit 20;", conn)
authors = pd.read_sql_query("select * from author;", conn)
authors

,id,full_name,first_name,last_name,middle_name,email,affiliation
0,1,"Benbya, Hind",Hind,Benbya,None,h.benbya@montpellier-bs.com,"Technology and Innovation Management, Montpell..."
1,2,Ning Nan,Ning,Nan,None,ning.nan@sauder.ubc.ca,"Sauder School of Business, University of Briti..."
2,3,"Tanriverdi, Hüseyin",Hüseyin,Tanriverdi,None,Huseyin.Tanriverdi@mccombs.utexas.edu,"McCombs School of Business, The University of ..."
3,4,Youngjin Yoo,Youngjin,Yoo,None,yxy23@case.edu,"Department of Design & Innovation, The Weather..."
4,5,"Pentland, Brian T.",Brian,Pentland,T.,pentlan2@msu.edu,"Eli Broad College of Business, Michigan State ..."
...,...,...,...,...,...,...,...
6361,6362,"Lockman, Abe",Abe,Lockman,None,nan,"Assistant Professor, Department of Computer Sc..."
6362,6363,"Shalev, Jacob",Jacob,Shalev,None,nan,"Graduate School of Business Administration, Ne..."
6363,6364,"Clark, Steven J.",Steven,Clark,J.,nan,"Electronics Engineer, Software Support Center,..."
6364,6365,"Norman, Morton",Morton,Norman,None,nan,"Assistant Professor, Department of Statistics ..."


In [3]:
# Split affiliation into columns by ',' indicating as features
affiliation_parse_raw = authors['affiliation'].str.split(",",expand = True)
# Convert to lower letters
affiliation_parse_lower = affiliation_parse_raw.apply(lambda x: x.astype(str).str.lower())
affiliation_parse_lower

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,technology and innovation management,montpellier business school,2300 avenue des moulins,montpellier 34185 cedex 4,france,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
1,sauder school of business,university of british columbia,2053 main mall,vancouver v6t 1z2 british columbia,canada,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
2,mccombs school of business,the university of texas at austin austin,tx 78712 u.s.a.,none,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
3,department of design & innovation,the weatherhead school of management,case western reserve university,10900 euclid avenue,cleveland,ohio 44106-7235 u.s.a. | warwick business school,university of warwick,coventry cv4 7rl united kingdom,none,none,...,none,none,none,none,none,none,none,none,none,none
4,eli broad college of business,michigan state university,east lansing,mi 48864 u.s.a.,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,assistant professor,department of computer science,rutgers university,none,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
6362,graduate school of business administration,new york university,none,none,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
6363,electronics engineer,software support center,warner robins air logistics center,none,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none
6364,assistant professor,department of statistics and computer informa...,baruch college,city university of new york,none,none,none,none,none,none,...,none,none,none,none,none,none,none,none,none,none


In [4]:
# Init a feature dataframe having same col and row as affiliation_parse_lower
affiliation_feature = pd.DataFrame().reindex_like(affiliation_parse_lower)
affiliation_feature

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# read institute list and country list
institutes = ['school', 'university', 'department','college','management','systems','center','institute']

countries = ['u.s.a.', 'united kingdom','canada', 'france','netherlands','norway','switzerland','sweden','china','india','singapore','south korea','australia','spain','hong kong','taiwan']

In [6]:
authors['country'] = 'nan'

# In order to test time complexity. 
# 16 countries list, 6366 rows for 1.41s, so if 100,000 items, then less than 25s
# If adding all countries, might be around 4 minutes. Can be far more less if we set up rules for issue where one author has more than one institutes.

# extract country firstly
def test(affiliation_parse_lower, countries, authors):
    for column in affiliation_parse_lower:
        for country in countries:
            authors['country'].loc[affiliation_parse_lower[column].str.contains(country)] = country

%timeit test(affiliation_parse_lower, countries, authors)

authors

/Users/harrywang/sandbox/pub-analyzer/venv/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1.68 s ± 145 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


,id,full_name,first_name,last_name,middle_name,email,affiliation,country
0,1,"Benbya, Hind",Hind,Benbya,None,h.benbya@montpellier-bs.com,"Technology and Innovation Management, Montpell...",france
1,2,Ning Nan,Ning,Nan,None,ning.nan@sauder.ubc.ca,"Sauder School of Business, University of Briti...",canada
2,3,"Tanriverdi, Hüseyin",Hüseyin,Tanriverdi,None,Huseyin.Tanriverdi@mccombs.utexas.edu,"McCombs School of Business, The University of ...",u.s.a.
3,4,Youngjin Yoo,Youngjin,Yoo,None,yxy23@case.edu,"Department of Design & Innovation, The Weather...",united kingdom
4,5,"Pentland, Brian T.",Brian,Pentland,T.,pentlan2@msu.edu,"Eli Broad College of Business, Michigan State ...",u.s.a.
...,...,...,...,...,...,...,...,...
6361,6362,"Lockman, Abe",Abe,Lockman,None,nan,"Assistant Professor, Department of Computer Sc...",nan
6362,6363,"Shalev, Jacob",Jacob,Shalev,None,nan,"Graduate School of Business Administration, Ne...",nan
6363,6364,"Clark, Steven J.",Steven,Clark,J.,nan,"Electronics Engineer, Software Support Center,...",nan
6364,6365,"Norman, Morton",Morton,Norman,None,nan,"Assistant Professor, Department of Statistics ...",nan


In [7]:
# if feature contain institutes keyword, label it as 1
for column in affiliation_parse_lower:
    for inst in institutes:
        affiliation_feature[column].loc[affiliation_parse_lower[column].str.contains(inst)] = 1
        
affiliation_feature

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6362,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6363,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# label all NaN value as '1' if there is some '1' on its right 
affiliation_feature = affiliation_feature.fillna(axis = 1, method='bfill')
affiliation_feature

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6362,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6363,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# combine all feature 1 together as institute
temp = pd.DataFrame().reindex_like(affiliation_parse_lower)
for column in affiliation_feature:
    temp[column] = affiliation_parse_raw.loc[affiliation_feature[column] == 1,[column]]
    
temp

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Technology and Innovation Management,Montpellier Business School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sauder School of Business,University of British Columbia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,McCombs School of Business,The University of Texas at Austin Austin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Department of Design & Innovation,The Weatherhead School of Management,Case Western Reserve University,10900 Euclid Avenue,Cleveland,Ohio 44106-7235 U.S.A. | Warwick Business School,University of Warwick,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Eli Broad College of Business,Michigan State University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,Assistant Professor,Department of Computer Science,Rutgers University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6362,Graduate School of Business Administration,New York University,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6363,Electronics Engineer,Software Support Center,Warner Robins Air Logistics Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,Assistant Professor,Department of Statistics and Computer Informa...,Baruch College,City University of New York,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# combine all feature 1 together as institute
temp = temp + ","
temp = temp.fillna('')
authors['institute'] = ''
for column in temp:
    authors['institute'] = authors['institute'] + temp[column]
    
authors['institute'] = authors['institute'].str.strip(',')
authors

,id,full_name,first_name,last_name,middle_name,email,affiliation,country,institute
0,1,"Benbya, Hind",Hind,Benbya,None,h.benbya@montpellier-bs.com,"Technology and Innovation Management, Montpell...",france,"Technology and Innovation Management, Montpell..."
1,2,Ning Nan,Ning,Nan,None,ning.nan@sauder.ubc.ca,"Sauder School of Business, University of Briti...",canada,"Sauder School of Business, University of Briti..."
2,3,"Tanriverdi, Hüseyin",Hüseyin,Tanriverdi,None,Huseyin.Tanriverdi@mccombs.utexas.edu,"McCombs School of Business, The University of ...",u.s.a.,"McCombs School of Business, The University of ..."
3,4,Youngjin Yoo,Youngjin,Yoo,None,yxy23@case.edu,"Department of Design & Innovation, The Weather...",united kingdom,"Department of Design & Innovation, The Weather..."
4,5,"Pentland, Brian T.",Brian,Pentland,T.,pentlan2@msu.edu,"Eli Broad College of Business, Michigan State ...",u.s.a.,"Eli Broad College of Business, Michigan State ..."
...,...,...,...,...,...,...,...,...,...
6361,6362,"Lockman, Abe",Abe,Lockman,None,nan,"Assistant Professor, Department of Computer Sc...",nan,"Assistant Professor, Department of Computer Sc..."
6362,6363,"Shalev, Jacob",Jacob,Shalev,None,nan,"Graduate School of Business Administration, Ne...",nan,"Graduate School of Business Administration, Ne..."
6363,6364,"Clark, Steven J.",Steven,Clark,J.,nan,"Electronics Engineer, Software Support Center,...",nan,"Electronics Engineer, Software Support Center,..."
6364,6365,"Norman, Morton",Morton,Norman,None,nan,"Assistant Professor, Department of Statistics ...",nan,"Assistant Professor, Department of Statistics ..."


In [11]:
# combine all feature NaN together as address
temp = pd.DataFrame().reindex_like(affiliation_parse_lower)
for column in affiliation_feature:
    temp[column] = affiliation_parse_raw.loc[affiliation_feature[column].isna(), [column]]
temp

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,NaN,2300 Avenue des Moulins,Montpellier 34185 Cedex 4,FRANCE,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,NaN,NaN,2053 Main Mall,Vancouver V6T 1Z2 British Columbia,CANADA,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,NaN,NaN,TX 78712 U.S.A.,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coventry CV4 7RL UNITED KINGDOM,None,None,...,None,None,None,None,None,None,None,None,None,None
4,NaN,NaN,East Lansing,MI 48864 U.S.A.,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# combine all feature NaN together as address
temp = temp + ","
temp = temp.fillna('')
authors['address'] = ''
for column in temp:
    authors['address'] = authors['address'] + temp[column]

authors['address'] = authors['address'].str.strip(',')
authors

,id,full_name,first_name,last_name,middle_name,email,affiliation,country,institute,address
0,1,"Benbya, Hind",Hind,Benbya,None,h.benbya@montpellier-bs.com,"Technology and Innovation Management, Montpell...",france,"Technology and Innovation Management, Montpell...","2300 Avenue des Moulins, Montpellier 34185 Ce..."
1,2,Ning Nan,Ning,Nan,None,ning.nan@sauder.ubc.ca,"Sauder School of Business, University of Briti...",canada,"Sauder School of Business, University of Briti...","2053 Main Mall, Vancouver V6T 1Z2 British Col..."
2,3,"Tanriverdi, Hüseyin",Hüseyin,Tanriverdi,None,Huseyin.Tanriverdi@mccombs.utexas.edu,"McCombs School of Business, The University of ...",u.s.a.,"McCombs School of Business, The University of ...",TX 78712 U.S.A.
3,4,Youngjin Yoo,Youngjin,Yoo,None,yxy23@case.edu,"Department of Design & Innovation, The Weather...",united kingdom,"Department of Design & Innovation, The Weather...",Coventry CV4 7RL UNITED KINGDOM
4,5,"Pentland, Brian T.",Brian,Pentland,T.,pentlan2@msu.edu,"Eli Broad College of Business, Michigan State ...",u.s.a.,"Eli Broad College of Business, Michigan State ...","East Lansing, MI 48864 U.S.A."
...,...,...,...,...,...,...,...,...,...,...
6361,6362,"Lockman, Abe",Abe,Lockman,None,nan,"Assistant Professor, Department of Computer Sc...",nan,"Assistant Professor, Department of Computer Sc...",
6362,6363,"Shalev, Jacob",Jacob,Shalev,None,nan,"Graduate School of Business Administration, Ne...",nan,"Graduate School of Business Administration, Ne...",
6363,6364,"Clark, Steven J.",Steven,Clark,J.,nan,"Electronics Engineer, Software Support Center,...",nan,"Electronics Engineer, Software Support Center,...",
6364,6365,"Norman, Morton",Morton,Norman,None,nan,"Assistant Professor, Department of Statistics ...",nan,"Assistant Professor, Department of Statistics ...",
